#Databricks notebook: Bronze - Risco de Fogo(Diário)

In [0]:
# ================================================================
# Databricks notebook: Bronze - INPE Fire Risk (Diário)
# ================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import xarray as xr
import datetime

spark = SparkSession.builder.getOrCreate()

In [0]:
# ================================================================
# Parâmetros recebidos via Databricks Job
# ================================================================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("table", "")
dbutils.widgets.text("path_raw", "")
dbutils.widgets.text("data_ref_carga", "")

catalog         = dbutils.widgets.get("catalog")
schema          = dbutils.widgets.get("schema")
table           = dbutils.widgets.get("table")
path_raw        = dbutils.widgets.get("path_raw")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

In [0]:
# ================================================================
# Validação do parâmetro de data
# ================================================================
if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado.")

print(f"📅 Data de referência: {data_ref_carga}")

# Remove hifens para montar o nome do arquivo
data_ref_fmt = data_ref_carga.replace("-", "")
file_name = f"INPE_FireRiskModel_2.2_FireRisk_{data_ref_fmt}.nc"
file_path = f"{path_raw}/{file_name}"

print(f"📂 Procurando arquivo: {file_path}")

In [0]:
# ================================================================
# Escrita na Tabela Bronze
# ================================================================
(
    df.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .partitionBy("data_ref_carga")
    .saveAsTable(f"{catalog}.{schema}.{table}")
)

print(f"💾 Dados gravados em: {catalog}.{schema}.{table}")
print("🚀 Job Bronze finalizado com sucesso!")